# **Programming Assessment \#4**

Names: Alyanna Abalos, Loben Tipan

More information on the assessment is found in our Canvas course.

# **Load Data**

*While you don't have to separate your code into blocks, it might be easier if you separated loading your data from actually implementation of your code. Consider placing all loading of data into the code block below.*

In [17]:
with open('count_1edit.txt', 'r', encoding='ISO-8859-1') as file:
    data_count_1edit = file.read()

with open('spell_errors.txt', 'r', encoding='ISO-8859-1') as file:
    data_spell_errors = file.read()


raining: rainning, raning
writings: writtings
disparagingly: disparingly
yellow: yello
four: forer, fours, fuore, fore*5, for*4
woods: woodes
hanging: haing
aggression: agression
looking: loking, begining, luing, look*2, locking, lucking, louk, looing, lookin, liking
eligible: eligble, elegable, eligable
electricity: electrisity, electricty*2, electrizity
scold: schold, skold
adaptable: adabtable
caned: canned, cained
immature: imature
shouldn't: shoudln, shouldnt
swivel: swival
appropriation: a


# **Error Model Implementation**

In [37]:
error_model = {}
for line in data_count_1edit.splitlines():
    parts = line.split('|')
    w = parts[0].strip() 
    
    if len(parts) < 2:
        continue

    c_split = parts[1].split()
    if len(c_split) < 2:
        continue

    c = c_split[0]
    count = int(c_split[1])
    
    error_model[(w, c)] = count

spell_errors = {}
for line in data_spell_errors.splitlines():
    correct_word, misspelled_words_part = line.split(":")
    misspelled_words = [word.strip() for word in misspelled_words_part.split(',')]
    spell_errors[correct_word.strip()] = misspelled_words


print(list(error_model.items())[:5], "\n\n")
print(list(spell_errors.items())[:5])

[(('e', 'i'), 917), (('a', 'e'), 856), (('i', 'e'), 771), (('e', 'a'), 749), (('a', 'i'), 559)] 


[('raining', ['rainning', 'raning']), ('writings', ['writtings']), ('disparagingly', ['disparingly']), ('yellow', ['yello']), ('four', ['forer', 'fours', 'fuore', 'fore*5', 'for*4'])]


In [39]:
# Error model
total_error_counts = sum(error_model.values())

error_model_probabilities = {pair: count / total_error_counts for pair, count in error_model.items()}


spell_errors_probabilities = {}
for correct_word, misspelled_words in spell_errors.items():
    for misspelled in misspelled_words:
        if (misspelled, correct_word) in error_model:
            if correct_word not in spell_errors_probabilities:
                spell_errors_probabilities[correct_word] = {}
            spell_errors_probabilities[correct_word][misspelled] = error_model[(misspelled, correct_word)] / total_error_counts

print(list(error_model_probabilities.items())[:5], "\n\n")
print(list(spell_errors_probabilities.items())[:5])

[(('e', 'i'), 0.02371715290709704), (('a', 'e'), 0.022139457893647838), (('i', 'e'), 0.01994103041589075), (('e', 'a'), 0.01937202565694186), (('a', 'i'), 0.014457893647837781)] 


[('I', {'a': 2.5863852679495136e-05, 'i': 2.5863852679495136e-05}), ('as', {'a': 0.0029484792054624456}), ('to', {'ot': 0.00012931926339747568}), ('be', {'b': 0.001138009517897786}), ('a', {'I': 2.5863852679495136e-05, 'ah': 7.759155803848541e-05, 'as': 0.0023794744465135526})]


# **Language Model Implementation**

*Again, you don't have to follow this directly, but consider placing your implementation of the model in the code block below.*